# Trading Strategy Dashboard - Classification

Comprehensive comparison of all models for CLASSIFICATION: LSTM, CNN, and ANN
Uses ETH-USD data with binary direction labels (up/down)
Includes classification metrics (accuracy, precision, recall, F1), IR2, equity curves, trade statistics, and Buy & Hold benchmark


In [ ]:
# Standard Library Imports
import os
import sys
import pickle

# Add project root to Python path
project_root = os.getcwd()
if project_root not in sys.path:
    sys.path.insert(0, project_root)

# Third Party Imports
import numpy as np
import pandas as pd
import polars as pl
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import vectorbt as vbt

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.float_format', lambda x: '%.4f' % x)


In [ ]:
# ============================================================================
# LOAD RESULTS FROM ALL MODELS
# ============================================================================

models = ['lstm', 'cnn', 'ann']
results_data = {}

for model_name in models:
    result_dir = f"results/classification/{model_name}"
    
    if os.path.exists(result_dir):
        results_data[model_name] = {
            'results_df': pd.read_csv(f"{result_dir}/results.csv") if os.path.exists(f"{result_dir}/results.csv") else None,
            'metrics': pd.read_csv(f"{result_dir}/comprehensive_metrics.csv") if os.path.exists(f"{result_dir}/comprehensive_metrics.csv") else None,
            'equity_curves': pd.read_csv(f"{result_dir}/equity_curves.csv") if os.path.exists(f"{result_dir}/equity_curves.csv") else None,
            'returns': pd.read_csv(f"{result_dir}/returns.csv") if os.path.exists(f"{result_dir}/returns.csv") else None,
            'trades': pd.read_csv(f"{result_dir}/all_trades.csv") if os.path.exists(f"{result_dir}/all_trades.csv") else None,
            'strategy_pf': None,
            'buyhold_pf': None
        }
        
        # Load portfolio objects if available
        if os.path.exists(f"{result_dir}/strategy_portfolio.pkl"):
            with open(f"{result_dir}/strategy_portfolio.pkl", "rb") as f:
                results_data[model_name]['strategy_pf'] = pickle.load(f)
        
        if os.path.exists(f"{result_dir}/buyhold_portfolio.pkl"):
            with open(f"{result_dir}/buyhold_portfolio.pkl", "rb") as f:
                results_data[model_name]['buyhold_pf'] = pickle.load(f)
        
        print(f"✓ Loaded {model_name.upper()} results")
    else:
        print(f"✗ {model_name.upper()} results not found")

print(f"\nLoaded results for {len(results_data)} model(s)")


In [ ]:
# ============================================================================
# COMPREHENSIVE METRICS COMPARISON TABLE
# ============================================================================

all_metrics = []
for model_name, data in results_data.items():
    if data['metrics'] is not None:
        metrics = data['metrics'].iloc[0].to_dict()
        all_metrics.append(metrics)

if all_metrics:
    metrics_df = pd.DataFrame(all_metrics)
    
    # Reorder columns for better readability
    column_order = [
        'model_type',
        'num_folds',
        'aggregated_test_ir2',
        'average_test_ir2',
        'average_val_ir2',
        'average_train_ir2',
        'average_test_accuracy',
        'average_test_precision',
        'average_test_recall',
        'average_test_f1',
        'strategy_total_return',
        'buyhold_total_return',
        'strategy_sharpe_ratio',
        'buyhold_sharpe_ratio',
        'strategy_max_drawdown',
        'buyhold_max_drawdown',
        'total_trades',
        'win_rate',
        'avg_return_per_trade',
        'avg_winning_trade',
        'avg_losing_trade',
        'largest_win',
        'largest_loss'
    ]
    
    # Select only columns that exist
    available_cols = [col for col in column_order if col in metrics_df.columns]
    metrics_df = metrics_df[available_cols]
    
    print("="*100)
    print("COMPREHENSIVE METRICS COMPARISON")
    print("="*100)
    print(metrics_df.to_string(index=False))
    
    # Format for display
    display_df = metrics_df.copy()
    if 'strategy_total_return' in display_df.columns:
        display_df['strategy_total_return'] = display_df['strategy_total_return'].apply(lambda x: f"{x:.2%}")
    if 'buyhold_total_return' in display_df.columns:
        display_df['buyhold_total_return'] = display_df['buyhold_total_return'].apply(lambda x: f"{x:.2%}")
    if 'win_rate' in display_df.columns:
        display_df['win_rate'] = display_df['win_rate'].apply(lambda x: f"{x:.2f}%")
    
    display(display_df)
else:
    print("No metrics data available")


In [ ]:
# ============================================================================
# EQUITY CURVES COMPARISON
# ============================================================================

fig = go.Figure()

# Add Buy & Hold (use first available)
buyhold_added = False
for model_name, data in results_data.items():
    if data['equity_curves'] is not None and not buyhold_added:
        equity_df = data['equity_curves']
        equity_df['date'] = pd.to_datetime(equity_df['date'])
        fig.add_trace(go.Scatter(
            x=equity_df['date'],
            y=equity_df['buyhold_equity'],
            mode='lines',
            name='Buy & Hold (Benchmark)',
            line=dict(color='gray', width=2, dash='dash')
        ))
        buyhold_added = True
        break

# Add strategy equity curves for each model
colors = {'lstm': 'blue', 'cnn': 'green', 'ann': 'orange'}
for model_name, data in results_data.items():
    if data['equity_curves'] is not None:
        equity_df = data['equity_curves']
        equity_df['date'] = pd.to_datetime(equity_df['date'])
        
        fig.add_trace(go.Scatter(
            x=equity_df['date'],
            y=equity_df['strategy_equity'],
            mode='lines',
            name=f"{model_name.upper()} Strategy",
            line=dict(color=colors.get(model_name, 'black'), width=2)
        ))

fig.update_layout(
    title='Equity Curves: All Models vs Buy & Hold',
    xaxis_title='Date',
    yaxis_title='Portfolio Value ($)',
    template='plotly_dark',
    hovermode='x unified',
    height=600,
    width=1200,
    legend=dict(yanchor="top", y=0.99, xanchor="left", x=0.01)
)

fig.show()


In [ ]:
# ============================================================================
# IR2 METRICS COMPARISON (Per Fold)
# ============================================================================

fig = make_subplots(
    rows=3, cols=1,
    subplot_titles=('Train IR2', 'Validation IR2', 'Test IR2'),
    vertical_spacing=0.1
)

colors = {'lstm': 'blue', 'cnn': 'green', 'ann': 'orange'}

for model_name, data in results_data.items():
    if data['results_df'] is not None:
        df = data['results_df']
        
        # Train IR2
        fig.add_trace(
            go.Scatter(
                x=df['fold'],
                y=df['train_IR2'],
                mode='lines+markers',
                name=f"{model_name.upper()} Train",
                line=dict(color=colors.get(model_name, 'black'), width=2),
                marker=dict(size=6)
            ),
            row=1, col=1
        )
        
        # Val IR2
        fig.add_trace(
            go.Scatter(
                x=df['fold'],
                y=df['val_IR2'],
                mode='lines+markers',
                name=f"{model_name.upper()} Val",
                line=dict(color=colors.get(model_name, 'black'), width=2),
                marker=dict(size=6)
            ),
            row=2, col=1
        )
        
        # Test IR2
        fig.add_trace(
            go.Scatter(
                x=df['fold'],
                y=df['test_IR2'],
                mode='lines+markers',
                name=f"{model_name.upper()} Test",
                line=dict(color=colors.get(model_name, 'black'), width=2),
                marker=dict(size=6)
            ),
            row=3, col=1
        )

fig.update_xaxes(title_text="Fold Number", row=3, col=1)
fig.update_yaxes(title_text="IR2", row=1, col=1)
fig.update_yaxes(title_text="IR2", row=2, col=1)
fig.update_yaxes(title_text="IR2", row=3, col=1)

fig.update_layout(
    title='IR2 Metrics Comparison Across Folds',
    template='plotly_dark',
    height=900,
    width=1200,
    showlegend=True
)

fig.show()


In [ ]:
# ============================================================================
# TRADE STATISTICS COMPARISON
# ============================================================================

trade_stats = []

for model_name, data in results_data.items():
    if data['trades'] is not None and len(data['trades']) > 0:
        trades_df = data['trades']
        
        stats = {
            'Model': model_name.upper(),
            'Total Trades': len(trades_df),
            'Winning Trades': len(trades_df[trades_df['return_pct'] > 0]),
            'Losing Trades': len(trades_df[trades_df['return_pct'] < 0]),
            'Win Rate (%)': (trades_df['return_pct'] > 0).sum() / len(trades_df) * 100,
            'Avg Return/Trade (%)': trades_df['return_pct'].mean(),
            'Avg Winning Trade (%)': trades_df[trades_df['return_pct'] > 0]['return_pct'].mean() if (trades_df['return_pct'] > 0).any() else 0,
            'Avg Losing Trade (%)': trades_df[trades_df['return_pct'] < 0]['return_pct'].mean() if (trades_df['return_pct'] < 0).any() else 0,
            'Largest Win (%)': trades_df['return_pct'].max(),
            'Largest Loss (%)': trades_df['return_pct'].min(),
            'Total Return (%)': trades_df['return_pct'].sum()
        }
        trade_stats.append(stats)

if trade_stats:
    trade_stats_df = pd.DataFrame(trade_stats)
    
    print("="*100)
    print("TRADE STATISTICS COMPARISON")
    print("="*100)
    print(trade_stats_df.to_string(index=False))
    
    # Visualize win rates
    fig = go.Figure()
    
    fig.add_trace(go.Bar(
        x=trade_stats_df['Model'],
        y=trade_stats_df['Win Rate (%)'],
        name='Win Rate',
        marker_color='green',
        text=trade_stats_df['Win Rate (%)'].apply(lambda x: f"{x:.1f}%"),
        textposition='outside'
    ))
    
    fig.update_layout(
        title='Win Rate Comparison Across Models',
        xaxis_title='Model',
        yaxis_title='Win Rate (%)',
        template='plotly_dark',
        height=500,
        width=800
    )
    
    fig.show()
    
    # Visualize number of trades
    fig = go.Figure()
    
    fig.add_trace(go.Bar(
        x=trade_stats_df['Model'],
        y=trade_stats_df['Total Trades'],
        name='Total Trades',
        marker_color='blue',
        text=trade_stats_df['Total Trades'],
        textposition='outside'
    ))
    
    fig.update_layout(
        title='Number of Trades Comparison',
        xaxis_title='Model',
        yaxis_title='Number of Trades',
        template='plotly_dark',
        height=500,
        width=800
    )
    
    fig.show()
else:
    print("No trade data available")


In [ ]:
# ============================================================================
# RETURNS DISTRIBUTION COMPARISON
# ============================================================================

fig = go.Figure()

colors = {'lstm': 'blue', 'cnn': 'green', 'ann': 'orange'}

for model_name, data in results_data.items():
    if data['returns'] is not None:
        returns_df = data['returns']
        returns_df['date'] = pd.to_datetime(returns_df['date'])
        
        fig.add_trace(go.Scatter(
            x=returns_df['date'],
            y=returns_df['strategy_returns'] * 100,  # Convert to percentage
            mode='lines',
            name=f"{model_name.upper()} Returns",
            line=dict(color=colors.get(model_name, 'black'), width=1.5),
            opacity=0.7
        ))

# Add Buy & Hold returns (from first available)
for model_name, data in results_data.items():
    if data['returns'] is not None:
        returns_df = data['returns']
        returns_df['date'] = pd.to_datetime(returns_df['date'])
        
        fig.add_trace(go.Scatter(
            x=returns_df['date'],
            y=returns_df['buyhold_returns'] * 100,
            mode='lines',
            name='Buy & Hold Returns',
            line=dict(color='gray', width=2, dash='dash')
        ))
        break

fig.update_layout(
    title='Daily Returns Comparison: All Models vs Buy & Hold',
    xaxis_title='Date',
    yaxis_title='Returns (%)',
    template='plotly_dark',
    hovermode='x unified',
    height=600,
    width=1200
)

fig.show()


In [ ]:
# ============================================================================
# PERFORMANCE SUMMARY TABLE
# ============================================================================

if all_metrics:
    summary_cols = [
        'model_type',
        'aggregated_test_ir2',
        'strategy_total_return',
        'buyhold_total_return',
        'strategy_sharpe_ratio',
        'strategy_max_drawdown',
        'total_trades',
        'win_rate'
    ]
    
    summary_df = metrics_df[[col for col in summary_cols if col in metrics_df.columns]].copy()
    
    # Format for display
    if 'strategy_total_return' in summary_df.columns:
        summary_df['strategy_total_return'] = summary_df['strategy_total_return'].apply(lambda x: f"{x:.2%}")
    if 'buyhold_total_return' in summary_df.columns:
        summary_df['buyhold_total_return'] = summary_df['buyhold_total_return'].apply(lambda x: f"{x:.2%}")
    if 'strategy_max_drawdown' in summary_df.columns:
        summary_df['strategy_max_drawdown'] = summary_df['strategy_max_drawdown'].apply(lambda x: f"{x:.2%}")
    if 'win_rate' in summary_df.columns:
        summary_df['win_rate'] = summary_df['win_rate'].apply(lambda x: f"{x:.2f}%")
    
    print("="*100)
    print("PERFORMANCE SUMMARY")
    print("="*100)
    print(summary_df.to_string(index=False))
    
    # Rank models by aggregated test IR2
    if 'aggregated_test_ir2' in metrics_df.columns:
        ranked = metrics_df.sort_values('aggregated_test_ir2', ascending=False)
        print("\n" + "="*100)
        print("MODEL RANKING (by Aggregated Test IR2)")
        print("="*100)
        for idx, row in ranked.iterrows():
            print(f"{idx+1}. {row['model_type']}: {row['aggregated_test_ir2']:.4f}")
else:
    print("No summary data available")


In [ ]:
# ============================================================================
# DRAWDOWN ANALYSIS
# ============================================================================

if all_metrics:
    fig = go.Figure()
    
    # Max drawdown comparison
    models_list = metrics_df['model_type'].values
    strategy_dd = metrics_df['strategy_max_drawdown'].values * 100  # Convert to percentage
    buyhold_dd = metrics_df['buyhold_max_drawdown'].values * 100
    
    fig.add_trace(go.Bar(
        x=models_list,
        y=strategy_dd,
        name='Strategy Max Drawdown',
        marker_color='red',
        text=[f"{x:.2f}%" for x in strategy_dd],
        textposition='outside'
    ))
    
    fig.add_trace(go.Bar(
        x=models_list,
        y=buyhold_dd,
        name='Buy & Hold Max Drawdown',
        marker_color='gray',
        text=[f"{x:.2f}%" for x in buyhold_dd],
        textposition='outside'
    ))
    
    fig.update_layout(
        title='Maximum Drawdown Comparison',
        xaxis_title='Model',
        yaxis_title='Max Drawdown (%)',
        template='plotly_dark',
        barmode='group',
        height=500,
        width=1000
    )
    
    fig.show()
else:
    print("No drawdown data available")


In [ ]:
# ============================================================================
# SHARPE RATIO COMPARISON
# ============================================================================

if all_metrics:
    fig = go.Figure()
    
    models_list = metrics_df['model_type'].values
    strategy_sharpe = metrics_df['strategy_sharpe_ratio'].values
    buyhold_sharpe = metrics_df['buyhold_sharpe_ratio'].values
    
    fig.add_trace(go.Bar(
        x=models_list,
        y=strategy_sharpe,
        name='Strategy Sharpe Ratio',
        marker_color='green',
        text=[f"{x:.3f}" for x in strategy_sharpe],
        textposition='outside'
    ))
    
    fig.add_trace(go.Bar(
        x=models_list,
        y=buyhold_sharpe,
        name='Buy & Hold Sharpe Ratio',
        marker_color='gray',
        text=[f"{x:.3f}" for x in buyhold_sharpe],
        textposition='outside'
    ))
    
    fig.update_layout(
        title='Sharpe Ratio Comparison',
        xaxis_title='Model',
        yaxis_title='Sharpe Ratio',
        template='plotly_dark',
        barmode='group',
        height=500,
        width=1000
    )
    
    fig.show()
else:
    print("No Sharpe ratio data available")

# ============================================================================
# CLASSIFICATION METRICS COMPARISON
# ============================================================================

if all_metrics and 'average_test_accuracy' in metrics_df.columns:
    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=('Accuracy', 'Precision', 'Recall', 'F1 Score'),
        vertical_spacing=0.15,
        horizontal_spacing=0.1
    )
    
    models_list = metrics_df['model_type'].values
    
    # Accuracy
    if 'average_test_accuracy' in metrics_df.columns:
        fig.add_trace(
            go.Bar(
                x=models_list,
                y=metrics_df['average_test_accuracy'].values * 100,
                name='Accuracy',
                marker_color='blue',
                text=[f"{x:.2f}%" for x in metrics_df['average_test_accuracy'].values * 100],
                textposition='outside',
                showlegend=False
            ),
            row=1, col=1
        )
    
    # Precision
    if 'average_test_precision' in metrics_df.columns:
        fig.add_trace(
            go.Bar(
                x=models_list,
                y=metrics_df['average_test_precision'].values * 100,
                name='Precision',
                marker_color='green',
                text=[f"{x:.2f}%" for x in metrics_df['average_test_precision'].values * 100],
                textposition='outside',
                showlegend=False
            ),
            row=1, col=2
        )
    
    # Recall
    if 'average_test_recall' in metrics_df.columns:
        fig.add_trace(
            go.Bar(
                x=models_list,
                y=metrics_df['average_test_recall'].values * 100,
                name='Recall',
                marker_color='orange',
                text=[f"{x:.2f}%" for x in metrics_df['average_test_recall'].values * 100],
                textposition='outside',
                showlegend=False
            ),
            row=2, col=1
        )
    
    # F1 Score
    if 'average_test_f1' in metrics_df.columns:
        fig.add_trace(
            go.Bar(
                x=models_list,
                y=metrics_df['average_test_f1'].values * 100,
                name='F1 Score',
                marker_color='red',
                text=[f"{x:.2f}%" for x in metrics_df['average_test_f1'].values * 100],
                textposition='outside',
                showlegend=False
            ),
            row=2, col=2
        )
    
    fig.update_yaxes(title_text="Score (%)", row=1, col=1)
    fig.update_yaxes(title_text="Score (%)", row=1, col=2)
    fig.update_yaxes(title_text="Score (%)", row=2, col=1)
    fig.update_yaxes(title_text="Score (%)", row=2, col=2)
    
    fig.update_layout(
        title='Classification Metrics Comparison',
        template='plotly_dark',
        height=700,
        width=1200
    )
    
    fig.show()
else:
    print("No classification metrics data available")


In [ ]:
# ============================================================================
# EXPORT COMPREHENSIVE SUMMARY
# ============================================================================

if all_metrics:
    # Save combined metrics
    os.makedirs("results/summary", exist_ok=True)
    metrics_df.to_csv("results/summary/all_models_metrics_classification.csv", index=False)
    print("✓ Saved combined metrics to results/summary/all_models_metrics_classification.csv")
    
    # Save performance summary
    if 'summary_df' in locals():
        summary_df.to_csv("results/summary/performance_summary.csv", index=False)
        print("✓ Saved performance summary to results/summary/performance_summary.csv")
    
    # Save trade statistics
    if 'trade_stats_df' in locals():
        trade_stats_df.to_csv("results/summary/trade_statistics.csv", index=False)
        print("✓ Saved trade statistics to results/summary/trade_statistics.csv")
    
    print("\nAll summary data exported to results/summary/")
else:
    print("No data to export")
